# Extraction (The Raw Input)

## Imports

In [5]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt

## Data Ingestion (Step 1.1)

First, we define our parameters. We will pull over 10 years of daily data for 'SPY' (S&P 500 ETF) and 'VIX' (Cboe Volatility Index).

In [3]:
# Define parameters
start_date = '2010-01-01'
end_date = dt.date.today().strftime('%Y-%m-%d')
tickers = ['SPY', '^VIX']

# Pull data
raw_data = yf.download(tickers, start=start_date, end=end_date, auto_adjust=False)

[*********************100%***********************]  2 of 2 completed


## Data Merge & Refinement (Step 1.2)

The ```yfinance``` download creates a multi-index DataFrame. We only care about the 'Adj Close' price for SPY and the 'Close' price for VIX (as VIX is not a stock and 'Adj Close' is often NaN). We'll select these, rename them for clarity, and create our master DataFrame.

In [4]:
# Select the relevant columns
spy_data = raw_data['Adj Close']['SPY']
vix_data = raw_data['Close']['^VIX'] # VIX uses 'Close'

# Combine into a single DataFrame
master_df = pd.concat([spy_data, vix_data], axis=1)
master_df.columns = ['SPY_Close', 'VIX_Close']

# Save the raw merged file as a checkpoint
master_df.to_csv('SPY_VIX_Raw.csv')

print("Master DataFrame head:")
print(master_df.head())

Master DataFrame head:
            SPY_Close  VIX_Close
Date                            
2010-01-04  85.279182  20.040001
2010-01-05  85.504944  19.350000
2010-01-06  85.565170  19.160000
2010-01-07  85.926353  19.059999
2010-01-08  86.212273  18.129999


## Initial Check (Step 1.3)

We run ```isnull().sum()``` to diagnose gaps. We expect to see missing data, as markets are closed on holidays (e.g., Christmas) and weekends, but ```yfinance``` typically only provides trading-day data. Any NaNs found here are likely from data-source inconsistencies.

In [6]:
# Run the initial null check
print("Missing values before transformation:")
print(master_df.isnull().sum())

Missing values before transformation:
SPY_Close    0
VIX_Close    0
dtype: int64


# Part 2: Transformation (Data Resilience)

## NaN Imputation (Step 2.1)

Financial time-series data must be continuous. A missing value (NaN) implies the market was open but data is missing. We use 'forward-fill' (```ffill```) to fill these gaps. This method assumes the 'last known price' is the most accurate representation for a non-trading day or brief data gap, which is standard practice."

In [8]:
# Forward-fill missing values
master_df.ffill(inplace=True)

# Run a final check to confirm all gaps are filled
print("\nMissing values after ffill:")
print(master_df.isnull().sum())


Missing values after ffill:
SPY_Close    0
VIX_Close    0
dtype: int64


## Feature Engineering (Trend & Returns) (Step 2.2)

We engineer new features. First, we calculate daily log returns, which are standard in financial modeling. Second, we create a 20-Day Simple Moving Average (SMA_20) on the SPY closing price to model short-term trend.

In [9]:
# Calculate daily log returns for SPY
master_df['SPY_Log_Returns'] = np.log(master_df['SPY_Close'] / master_df['SPY_Close'].shift(1))

# Create 20-Day Simple Moving Average (SMA_20)
master_df['SMA_20'] = master_df['SPY_Close'].rolling(window=20).mean()

## Feature Engineering (Risk) (Step 2.3)

Price alone is insufficient; we must model risk. We will create a 20-Day rolling volatility, which is the standard deviation of the daily log returns. This feature quantifies market risk over the last 20 trading days.

In [12]:
# Create 20-Day Volatility (std dev of log returns)
master_df['Volatility_20'] = master_df['SPY_Log_Returns'].rolling(window=20).std()

print("\nDataFrame head after feature engineering:")
print(master_df.head(25)) # Show 25 to see NaNs


DataFrame head after feature engineering:
            SPY_Close  VIX_Close  SPY_Log_Returns     SMA_20  Volatility_20
Date                                                                       
2010-01-04  85.279182  20.040001              NaN        NaN            NaN
2010-01-05  85.504944  19.350000         0.002644        NaN            NaN
2010-01-06  85.565170  19.160000         0.000704        NaN            NaN
2010-01-07  85.926353  19.059999         0.004212        NaN            NaN
2010-01-08  86.212273  18.129999         0.003322        NaN            NaN
2010-01-11  86.332672  17.549999         0.001396        NaN            NaN
2010-01-12  85.527504  18.250000        -0.009370        NaN            NaN
2010-01-13  86.249908  17.850000         0.008411        NaN            NaN
2010-01-14  86.483192  17.629999         0.002701        NaN            NaN
2010-01-15  85.512489  17.910000        -0.011288        NaN            NaN
2010-01-19  86.581001  17.580000         0.01

## Clean Start (Step 2.4)

Our rolling window calculations (SMA and Volatility) created NaNs for the first 19 rows (since they need 20 days of data). These rows cannot be used for backtesting. We now drop them to create our final, clean, feature-rich dataset.

In [14]:
# Drop the initial NaN rows created by rolling windows
clean_df = master_df.dropna()

print("\nFinal clean DataFrame head:")
print(clean_df.head())

print("\nFinal null check on clean data:")
print(clean_df.isnull().sum())


Final clean DataFrame head:
            SPY_Close  VIX_Close  SPY_Log_Returns     SMA_20  Volatility_20
Date                                                                       
2010-02-02  83.059364  21.480000         0.012031  84.348005       0.010619
2010-02-03  82.645515  21.600000        -0.004995  84.205033       0.010606
2010-02-04  80.094589  26.080000        -0.031352  83.931504       0.012480
2010-02-05  80.260147  26.110001         0.002065  83.648194       0.012421
2010-02-08  79.680672  26.510000        -0.007246  83.321614       0.012344

Final null check on clean data:
SPY_Close          0
VIX_Close          0
SPY_Log_Returns    0
SMA_20             0
Volatility_20      0
dtype: int64


# Part 3: Load & Documentation (The Deliverable)

## Load (Step 3.1)

Finally, we save our clean, feature-rich DataFrame to a new CSV file. This file is now ready for direct input into a backtesting engine (Project 2) or any machine learning model.

In [16]:
# Save the final file
clean_df.to_csv('SPY_Clean_Featured_Data.csv')

print("\n'SPY_Clean_Featured_Data.csv' saved successfully.")


'SPY_Clean_Featured_Data.csv' saved successfully.
